In [2]:
# So ... each agent also has a type ideally

#Thought: with the type, we could set it within the agent itself.
#That way only need to pass the type?
#
#I guess that agents don't have access to the model?

SyntaxError: invalid syntax (2988153278.py, line 3)

In [41]:
# https://stackoverflow.com/questions/19620498/how-to-create-a-utility-class-correctly

In [42]:
DEBUG = False

def setup_p_txns(total_steps):
    # Mean at 0
    max_std_devs = 5 # 

    if DEBUG:
        # Note: scipy norm has stddev = 1, mean of 0
        # Below shows why we pick max_std_devs = 5
        p_txn = scipy.integrate.quad(
                scipy.stats.norm.pdf, -5, 5)[0] # quad() Returns: value, error
        print('In total we will capture this amount of the gaussian: ', p_txn)
    
    txn_probabilities = []

    std_dev_per_step = 2 * max_std_devs / total_steps

    for step in range(total_steps):
        p_txn = \
            scipy.integrate.quad(
                scipy.stats.norm.pdf, 
                step * std_dev_per_step, (step+1) * std_dev_per_step
            ) # returns: (integration) value, error
        txn_probabilities.append(p_txn[0])
    if DEBUG:
        txn_probabilities[0] = 1 # set p_txn to 1 to always transact
        txn_probabilities[-1] = 1
        txn_probabilities[1:-1] = 0
    return txn_probabilities

# Okay... so... 
We have agents...

Suspicious ones
Not suspicious ones


Transactions should have:


We also want to export the network of transactions

So... the data we want are... 

sender - receiver - amount - timestamp

parameters:

num of agents,
avg txn hr -> so at any given point, the likelihood of making a transaction

avg txn value
distribution of p per agent type 
starting balance




In [43]:
# export some... network graphs i guess....
# also want to make some classifieirs

In [44]:
# Model design
import agentpy as ap
import numpy as np

# Visualization
import seaborn as sns
import pandas as pd

import scipy.stats
import datetime
import matplotlib.pyplot as plt

In [99]:
# work
rng = np.random.default_rng(42)
rng.random()
rng.normal(loc=0, scale=1, size=1)

array([-1.03998411])

In [123]:
class BankAgent(ap.Agent):
    def setup(self):
        self.txn_amt_rng = None 
        self.txn_probabilities = None 
        self.type = None 
        self.acct_balance = None 

        self.txns = pd.DataFrame(
                columns=['timestep', 'sender_id', 'sender_type', 
                 'receiver_id', 'receiver_type', 'amount', 'acct_balance']
        )
        self.txn_amts = None

    def setup_txn_amts(self, mean=100, stddev=10, total_steps=0):
        self.txn_amts = self.txn_amt_rng.normal(
            loc=mean, scale=stddev, 
            size=total_steps)

    def transact(self, timestep):
        #shift = self.mean * 4 # WARNING hardcoded 4
        p_txn = self.txn_probabilities[timestep-1]
        if np.random.random() < p_txn and self.acct_balance > 0:
            # randomly choose partner
            partner = self.model.agents.random()
            # randomly choose amount
            amount = self.txn_amts[timestep]

            print('transacted. timestep', timestep, f'ptxn: {p_txn:.3f}',
            'with partner: ', *partner.id, 'new balance: ', self.acct_balance)

            # calculations
            partner.acct_balance += amount
            self.acct_balance -= amount

            # record
            self.txns.loc[len(self.txns.index)] = [
                timestep, self.id, self.type, *partner.id, partner.type,
                amount, self.acct_balance]

# parameters

In [136]:
        #self.txn_value_rng = p.rng
        #self.txn_probabilities = p.txn_probabilities
        ##self.p.type = 'default'
        #self.type = self.p.type
        #self.acct_balance = self.p.init_acct_balance

MEAN_TXN_HRS = {'normal':14, 
                'suspicious':22}
MINS_PER_STEP = 15

MEAN_TXN_AMOUNTS = {'normal':250, 
                'suspicious':50}

NUM_AGENTS_PER_TYPE = {'normal': 10,
    'suspicious': 2
}

parameters = {
    'num_agents_per_type': NUM_AGENTS_PER_TYPE,
    'mean_txn_hrs': MEAN_TXN_HRS,
    'mean_txn_amounts ': MEAN_TXN_AMOUNTS,
    'starting_balance': 100,
    'seed': 42,
    'mins_per_step' : MINS_PER_STEP, # 1 hr
    'steps': int(24 * (60/MINS_PER_STEP)), # 24 hours * steps per hr
}

# For plotting
def timestep_to_time(timestep):
    date_and_time = datetime.datetime(2022, 10, 31, 0, 0, 0)
    time_elapsed = timestep * (parameters['mins_per_step'])
    time_change = datetime.timedelta(minutes=time_elapsed)
    new_time = date_and_time + time_change
    return new_time

In [137]:
a = {'asdf':1, 'dfg':2}
a.values()

dict_values([1, 2])

In [138]:
np.random.default_rng(42).random(10).round(3) * 1000

array([774., 439., 859., 697.,  94., 976., 761., 786., 128., 450.])

In [148]:
class BankModel(ap.Model):
    def setup(self):
        self.p_txns = setup_p_txns(self.p.steps) 
        num_agents = sum(self.p.num_agents_per_type.values())
        
        # Setup up rng to generate seeds for rngs for agents
        rng = np.random.default_rng(self.p.seed)
        agent_rng_seeds = rng.random(num_agents).round(3) * 1000
        agent_rng_seeds = np.array(agent_rng_seeds, dtype=int)

        list_agents = []
        for type in ['normal', 'suspicious']:

            hrs_mean = self.p.mean_txn_hrs[type]
            shift = hrs_mean * int(60 / self.p.mins_per_step)
            # shift in steps
            shifted = np.concatenate(
                (self.p_txns[-shift:], self.p_txns[:-shift]))
            agents = ap.AgentList(
                self, self.p.num_agents_per_type[type], BankAgent)

            agents.txn_probabilities = shifted
            agents.total_steps = self.p.steps 
            agents.acct_balance = 100
            agents.type = type

            list_agents.append(agents)

        print('here are all the agetns: ')
        print(list_agents)
        # make all the agents
        self.agents = ap.AgentList(self, list_agents)
        print('here are all the agetns: ')
        print(self.agents)

        # calcuations for txn values
        txn_amt_rngs = ap.AttrIter(
            [np.random.default_rng(seed) for seed in agent_rng_seeds])
        self.agents.txn_amt_rng = txn_amt_rngs

        for agent in self.agents:
            agent.setup_txn_amts(mean=100, stddev=10, total_steps=self.p.steps)
            print(agent.id)


    def step(self):
        self.agents.transact(self.t)

    def update(self):
        #self.record(foo_bar)
        total_txns = 0
        for agent in self.agents:
            total_txns += len(agent.txns.shape[0])
        self.record('num txns (across all agents)', total_txns)

    def end(self):
        self.agents.record('txns')

In [149]:
np.random.default_rng(42).normal()

0.30471707975443135

In [150]:
model = BankModel(parameters)
results = model.run()

here are all the agetns: 
[AgentList (10 objects), AgentList (2 objects)]
here are all the agetns: 
AgentList (2 objects)
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[11, 12]
Completed: 1 stepstransacted. timestep 2 ptxn: 0.025 with partner:  [11, 12] new balance:  100
Completed: 8 stepstransacted. timestep 9 ptxn: 0.009 with partner:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] new balance:  202.06340463336144
Completed: 66 stepstransacted. timestep 67 ptxn: 0.023 with partner:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] new balance:  210.6678496245907
Completed: 67 stepstransacted. timestep 68 ptxn: 0.020 with partner:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] new balance:  327.28299152154574
transacted. timestep 68 ptxn: 0.020 with partner:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] new balance:  439.6125781854531
Completed: 70 stepstransacted. timestep 71 ptxn: 0.013 with partner:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] new balance:  542.0109655258894
Completed: 96 steps
Run time: 0:00:00.100184
Simulation finished


# run model